In [ ]:
# Configuration de Gemini
import google.generativeai as genai
import json
import pandas as pd
from IPython.display import display, Markdown
import time
import os
from dotenv import load_dotenv
# Charger les variables d'environnement depuis le fichier .env
load_dotenv()

# Récupérer la clé API Gemini depuis les variables d'environnement
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

if not GEMINI_API_KEY:
    raise ValueError(
        "❌ Clé API Gemini introuvable.\n"
        "➡️ Vérifie que :\n"
        "1. Le fichier .env existe\n"
        "2. Il contient la ligne : GEMINI_API_KEY=ta_cle_api\n"
        "3. Le fichier .env est bien chargé"
    )

# Configurer Gemini
genai.configure(api_key=GEMINI_API_KEY)

# Utiliser le modèle Gemini
model = genai.GenerativeModel("gemini-1.5-flash")

print("✅ Gemini configuré avec succès !")

✅ Gemini configuré avec succès !


In [11]:
# Charger les FDs de la Tâche 1
with open('../results/task1_all_fds.json', 'r') as f:
    all_fds = json.load(f)

print("Datasets disponibles:", list(all_fds.keys()))
print("\nNombre de FDs par dataset:")
for name, fds in all_fds.items():
    print(f"  - {name}: {len(fds)} FDs")

Datasets disponibles: ['abalone', 'balance-scale', 'breast-cancer-wisconsin', 'bridges', 'chess', 'echocardiogram', 'hepatitis', 'iris', 'nursery']

Nombre de FDs par dataset:
  - abalone: 137 FDs
  - balance-scale: 1 FDs
  - breast-cancer-wisconsin: 46 FDs
  - bridges: 144 FDs
  - chess: 1 FDs
  - echocardiogram: 527 FDs
  - hepatitis: 8296 FDs
  - iris: 4 FDs
  - nursery: 1 FDs


## 1. Comprendre les datasets

Avant d'analyser les FDs, on doit comprendre ce que représentent les attributs.

### Dataset IRIS (fleurs)
| Colonne | Signification |
|---------|---------------|
| 1 | sepal_length (longueur sépale) |
| 2 | sepal_width (largeur sépale) |
| 3 | petal_length (longueur pétale) |
| 4 | petal_width (largeur pétale) |
| 5 | class (espèce: Setosa, Versicolor, Virginica) |

### Dataset BRIDGES (ponts de Pittsburgh)
| Colonne | Signification |
|---------|---------------|
| IDENTIF (E1, E2...) | Identifiant unique du pont |
| RIVER | Rivière traversée (A, M, O) |
| ERECTED | Année de construction |
| PURPOSE | Usage (WALK, RR, HIGHWAY) |
| MATERIAL | Matériau (WOOD, IRON, STEEL) |
| SPAN | Portée (SHORT, MEDIUM, LONG) |
| TYPE | Type de pont (ARCH, SUSPEN, etc.) |

### Dataset BREAST-CANCER-WISCONSIN
| Colonne | Signification |
|---------|---------------|
| ID | ID du patient |
| 1-9 | Caractéristiques cellulaires (taille, forme, uniformité, etc.) |
| class | Diagnostic (2=bénin, 4=malin) |

### Dataset ABALONE (coquillages)
| Colonne | Signification |
|---------|---------------|
| 1 | Sex (M, F, I=infant) |
| 2 | Length |
| 3 | Diameter |
| 4 | Height |
| 5 | Whole weight |
| 6 | Shucked weight |
| 7 | Viscera weight |
| 8 | Shell weight |
| 9 | Rings (âge) |

### Dataset NURSERY (évaluation de garderies)
| Colonne | Signification |
|---------|---------------|
| parents | Occupation des parents |
| has_nurs | A une garderie |
| form | Forme de la famille |
| children | Nombre d'enfants |
| housing | Situation de logement |
| finance | Situation financière |
| social | Conditions sociales |
| health | Santé |
| class | Recommandation finale |

## 2. Sélection des FDs à analyser

On va sélectionner des FDs de différents types :
- **Plausibles** : qui semblent avoir du sens
- **Suspectes** : qui semblent accidentelles ou problématiques

In [12]:
# FDs sélectionnées pour l'analyse

fds_to_analyze = [
    # === FDs PLAUSIBLES ===
    {
        'id': 1,
        'dataset': 'iris',
        'fd': 'petal_length, petal_width, sepal_length → species',
        'context': 'Dataset de fleurs iris avec mesures de pétales et sépales pour classifier 3 espèces.',
        'category_expected': 'plausible'
    },
    {
        'id': 2,
        'dataset': 'bridges',
        'fd': 'MATERIAL, SPAN → TYPE',
        'context': 'Dataset de ponts de Pittsburgh avec matériau (WOOD, IRON, STEEL), portée (SHORT, MEDIUM, LONG) et type de pont.',
        'category_expected': 'plausible'
    },
    {
        'id': 3,
        'dataset': 'breast-cancer',
        'fd': 'cell_size, cell_shape → diagnosis',
        'context': 'Dataset médical avec caractéristiques cellulaires pour diagnostic de cancer (bénin/malin).',
        'category_expected': 'plausible'
    },
    
    # === FDs SUSPECTES ===
    {
        'id': 4,
        'dataset': 'bridges',
        'fd': 'E1 → MATERIAL, SPAN, TYPE, RIVER, ERECTED (E1 est l\'identifiant unique du pont)',
        'context': 'Dataset de ponts où E1 est le numéro d\'identification unique de chaque pont.',
        'category_expected': 'degenerate'
    },
    {
        'id': 5,
        'dataset': 'nursery',
        'fd': 'parents, has_nurs, form, children, housing, finance, social → recommendation',
        'context': 'Dataset d\'évaluation de garderies avec 7 critères qui déterminent la recommandation.',
        'category_expected': 'overfitted'
    },
    {
        'id': 6,
        'dataset': 'abalone',
        'fd': 'shell_weight, viscera_weight, whole_weight, sex → diameter',
        'context': 'Dataset de coquillages (abalones) avec mesures de poids et dimensions.',
        'category_expected': 'accidental'
    }
]

print(f"Nombre de FDs à analyser: {len(fds_to_analyze)}")
print("\n=== FDs PLAUSIBLES ===")
for fd in fds_to_analyze[:3]:
    print(f"  [{fd['id']}] {fd['dataset']}: {fd['fd']}")
print("\n=== FDs SUSPECTES ===")
for fd in fds_to_analyze[3:]:
    print(f"  [{fd['id']}] {fd['dataset']}: {fd['fd']}")

Nombre de FDs à analyser: 6

=== FDs PLAUSIBLES ===
  [1] iris: petal_length, petal_width, sepal_length → species
  [2] bridges: MATERIAL, SPAN → TYPE
  [3] breast-cancer: cell_size, cell_shape → diagnosis

=== FDs SUSPECTES ===
  [4] bridges: E1 → MATERIAL, SPAN, TYPE, RIVER, ERECTED (E1 est l'identifiant unique du pont)
  [5] nursery: parents, has_nurs, form, children, housing, finance, social → recommendation
  [6] abalone: shell_weight, viscera_weight, whole_weight, sex → diameter


## 3. Fonction pour interroger Gemini

In [13]:
def ask_gemini_about_fd(fd_info):
    """
    Interroge Gemini sur une dépendance fonctionnelle.
    Retourne la classification et l'explication.
    """
    
    prompt = f"""
Tu es un expert en bases de données et en qualité des données.

Contexte du dataset "{fd_info['dataset']}":
{fd_info['context']}

Voici une dépendance fonctionnelle (FD) découverte par un algorithme:
    {fd_info['fd']}

Une dépendance fonctionnelle X → Y signifie que si deux lignes ont la même valeur pour X, elles doivent avoir la même valeur pour Y.

Question: Cette dépendance fonctionnelle a-t-elle du sens dans le monde réel?

Réponds EXACTEMENT dans ce format:
CLASSIFICATION: [une seule parmi: MEANINGFUL, ACCIDENTAL, ENCODING-BASED, DEGENERATE, OVERFITTED, UNLIKELY]
EXPLICATION: [une phrase d'explication]

Définitions:
- MEANINGFUL: La FD représente une vraie règle du monde réel
- ACCIDENTAL: La FD est vraie dans les données mais par coïncidence statistique
- ENCODING-BASED: L'information du RHS est encodée dans le LHS
- DEGENERATE: La FD est basée sur un identifiant unique (ID → tout)
- OVERFITTED: Trop d'attributs dans le LHS, c'est de la mémorisation
- UNLIKELY: La FD ne devrait pas exister logiquement
"""
    
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"ERREUR: {str(e)}"

print("✅ Fonction ask_gemini_about_fd() définie !")

✅ Fonction ask_gemini_about_fd() définie !


## 4. Interroger Gemini pour chaque FD

In [14]:
# Interroger Gemini pour chaque FD
gemini_responses = {}

print("="*80)
print("INTERROGATION DE GEMINI")
print("="*80)

for fd in fds_to_analyze:
    print(f"\n🔄 Analyse de la FD #{fd['id']} ({fd['dataset']})...")
    
    response = ask_gemini_about_fd(fd)
    gemini_responses[fd['id']] = response
    
    print(f"📝 Réponse de Gemini:")
    print(response)
    print("-" * 40)
    
    # Pause pour éviter de dépasser les limites de l'API
    time.sleep(2)

print("\n✅ Toutes les FDs ont été analysées par Gemini !")

INTERROGATION DE GEMINI

🔄 Analyse de la FD #1 (iris)...
📝 Réponse de Gemini:
ERREUR: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
]
----------------------------------------

🔄 Analyse de la FD #2 (bridges)...
📝 Réponse de Gemini:
ERREUR: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
]
----------------------------------------

🔄 Analyse de la FD #3 (breast-cancer)...
📝 Réponse de Gemini:
ERREUR: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
,

## 5. Parser les réponses de Gemini

In [ ]:
def parse_gemini_response(response_text):
    """
    Extrait la classification et l'explication de la réponse de Gemini.
    """
    lines = response_text.strip().split('\n')
    classification = "UNKNOWN"
    explanation = ""
    
    for line in lines:
        if 'CLASSIFICATION:' in line.upper():
            classification = line.split(':', 1)[1].strip() if ':' in line else "UNKNOWN"
        elif 'EXPLICATION:' in line.upper():
            explanation = line.split(':', 1)[1].strip() if ':' in line else ""
    
    return classification, explanation

# Parser toutes les réponses
llm_judgments = {}
for fd_id, response in gemini_responses.items():
    classification, explanation = parse_gemini_response(response)
    llm_judgments[fd_id] = {
        'classification': classification,
        'explanation': explanation,
        'raw_response': response
    }

print("Réponses parsées:")
for fd_id, judgment in llm_judgments.items():
    print(f"  FD #{fd_id}: {judgment['classification']}")

## 6. Notre jugement humain

In [ ]:
# Notre propre jugement (humain)

human_judgments = {
    1: {
        'classification': 'MEANINGFUL',
        'explanation': 'Les mesures physiques des fleurs sont utilisées en botanique pour la classification des espèces.'
    },
    2: {
        'classification': 'MEANINGFUL',
        'explanation': 'En ingénierie des ponts, le matériau et la portée dictent le type de structure nécessaire.'
    },
    3: {
        'classification': 'MEANINGFUL',
        'explanation': 'Les caractéristiques cellulaires sont effectivement utilisées pour le diagnostic médical.'
    },
    4: {
        'classification': 'DEGENERATE',
        'explanation': 'Un identifiant unique détermine toujours tous les autres attributs - c\'est trivial.'
    },
    5: {
        'classification': 'OVERFITTED',
        'explanation': '7 attributs pour déterminer 1 résultat = mémorisation, pas une vraie règle générale.'
    },
    6: {
        'classification': 'ACCIDENTAL',
        'explanation': 'Corrélation physique entre poids et dimensions, mais pas une règle déterministe.'
    }
}

print("✅ Jugements humains définis pour", len(human_judgments), "FDs")

## 7. Tableau de comparaison LLM vs Humain

In [ ]:
# Créer le tableau de comparaison

comparison_data = []
for fd in fds_to_analyze:
    fd_id = fd['id']
    llm = llm_judgments.get(fd_id, {})
    human = human_judgments.get(fd_id, {})
    
    llm_class = llm.get('classification', 'N/A')
    human_class = human.get('classification', 'N/A')
    
    # Normaliser pour comparaison
    llm_normalized = llm_class.upper().strip()
    human_normalized = human_class.upper().strip()
    
    agreement = 'OUI' if llm_normalized == human_normalized else 'NON'
    
    comparison_data.append({
        'ID': fd_id,
        'Dataset': fd['dataset'],
        'FD': fd['fd'][:45] + '...' if len(fd['fd']) > 45 else fd['fd'],
        'Gemini': llm_class,
        'Humain': human_class,
        'Accord': agreement
    })

comparison_df = pd.DataFrame(comparison_data)

print("\n" + "="*100)
print("TABLEAU DE COMPARAISON : GEMINI vs HUMAIN")
print("="*100)
display(comparison_df)

# Statistiques
agreements = sum(1 for d in comparison_data if d['Accord'] == 'OUI')
total = len(comparison_data)
print(f"\n📊 Statistiques:")
print(f"   - Accords: {agreements}/{total} ({100*agreements/total:.0f}%)")
print(f"   - Désaccords: {total - agreements}")

## 8. Analyse des désaccords

In [ ]:
# Afficher les détails des désaccords

print("\n" + "="*80)
print("ANALYSE DES DÉSACCORDS")
print("="*80)

disagreements = [d for d in comparison_data if d['Accord'] == 'NON']

if disagreements:
    for i, d in enumerate(disagreements, 1):
        fd_id = d['ID']
        print(f"\n--- Désaccord {i}: FD #{fd_id} ({d['Dataset']}) ---")
        print(f"FD: {fds_to_analyze[fd_id-1]['fd']}")
        print(f"\n  🤖 Gemini dit: {d['Gemini']}")
        print(f"     Explication: {llm_judgments[fd_id]['explanation']}")
        print(f"\n  👤 Humain dit: {d['Humain']}")
        print(f"     Explication: {human_judgments[fd_id]['explanation']}")
else:
    print("\n✅ Aucun désaccord ! Gemini et l'humain sont d'accord sur toutes les FDs.")

## 9. Sauvegarder les résultats

In [ ]:
# Sauvegarder le tableau de comparaison
comparison_df.to_csv('../results/task2_comparison.csv', index=False)
print("✅ Tableau sauvegardé: results/task2_comparison.csv")

# Sauvegarder les réponses complètes de Gemini
gemini_full = []
for fd in fds_to_analyze:
    fd_id = fd['id']
    gemini_full.append({
        'fd_id': fd_id,
        'dataset': fd['dataset'],
        'fd': fd['fd'],
        'gemini_classification': llm_judgments[fd_id]['classification'],
        'gemini_explanation': llm_judgments[fd_id]['explanation'],
        'gemini_raw': llm_judgments[fd_id]['raw_response'],
        'human_classification': human_judgments[fd_id]['classification'],
        'human_explanation': human_judgments[fd_id]['explanation']
    })

with open('../results/task2_gemini_responses.json', 'w', encoding='utf-8') as f:
    json.dump(gemini_full, f, indent=2, ensure_ascii=False)
print("✅ Réponses Gemini: results/task2_gemini_responses.json")

# Sauvegarder les prompts
prompts = [{'fd_id': fd['id'], 'dataset': fd['dataset'], 'fd': fd['fd'], 'context': fd['context']} for fd in fds_to_analyze]
with open('../prompts/task2_prompts.json', 'w', encoding='utf-8') as f:
    json.dump(prompts, f, indent=2, ensure_ascii=False)
print("✅ Prompts: prompts/task2_prompts.json")

## 10. Conclusion

### Points forts de Gemini :
- Comprend le contexte métier
- Détecte les FDs dégénérées (basées sur IDs)

### Points faibles de Gemini :
- Peut ne pas détecter les FDs sur-ajustées
- Raisonne sur le "devrait être" vs les données réelles

### Recommandation :
> Combiner LLM (filtre sémantique) + vérification algorithmique (taille LHS)

In [ ]:
print("\n" + "="*80)
print("RÉSUMÉ FINAL - TÂCHE 2")
print("="*80)
print(f"\n✅ FDs analysées: {len(fds_to_analyze)}")
print(f"✅ LLM utilisé: Google Gemini 1.5 Flash")
print(f"✅ Accords: {agreements}/{total}")
print(f"✅ Désaccords: {total - agreements}")
print(f"\n📁 Fichiers générés:")
print(f"   - results/task2_comparison.csv")
print(f"   - results/task2_gemini_responses.json")
print(f"   - prompts/task2_prompts.json")